In [1]:
import json
import w3lib.html
import random
import os
import datetime
import requests
import string
import calendar
import html
import locale 
import validators
from requests_html import HTML
from requests_html import HTMLSession

## Crawler

In [ ]:
SEED_CRAWL = 'https://globalnews.ca/wp-json/wp/v2/posts'

In [ ]:
class Crawler:
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
]

    #query_extract_all_href=QUERY_URL

    all_json_to_send = []
    
    def _init_(self):
        pass
    
    def parse(self, response):
        ''' do something with this parser '''
        self.all_extracted_href = []
        json_data = json.loads(response.html.html)
        for url_data in json_data:
           
            json_to_send = {}
            json_to_send["url"] = url_data["guid"]["rendered"]
            json_to_send["title"] = url_data["title"]["rendered"]
            json_to_send["date"] = url_data["date_gmt"]
            self.all_json_to_send.append(json_to_send)      
                     

    def start_request(self,url,header=dict()):
        session = HTMLSession()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        response = session.get(url)  
        return response
    
    def get_number_of_pages(self,url,header=dict()):
        session = HTMLSession()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        response = session.get(url)  
        return response.headers['X-WP-TotalPages']

In [ ]:
crawler = Crawler()
# Obtener el numero de paginas
number_of_pages = crawler.get_number_of_pages(SEED_CRAWL)
print(SEED_CRAWL, number_of_pages)

In [ ]:
all_data_crawler = []

In [ ]:
number_of_pages = 10
for number_of_page in range(1, number_of_pages):
    response = crawler.start_request(SEED_CRAWL+'?page='+str(number_of_page))
    print('Se obtuvo la pagina '+str(number_of_page))
    crawler.parse(response)
    data_crawler = crawler.all_json_to_send
    all_data_crawler = all_data_crawler + data_crawler

In [ ]:
all_data_crawler

In [ ]:
# Almacenar los datos en un archivo json
with open('canada_global_news_links.json', 'w') as f:
    json.dump(all_data_crawler, f)

## Scrapper

In [4]:
with open('canada_global_news_links.json') as json_file:
    data = json.load(json_file)

In [5]:
QUERY_CONTENT =  "//article[contains(@class, 'l-article__text') and contains(@class, 'js-story-text')]/p"

QUERY_CONTENT_JOURNALIST =  "//div[@class='c-byline__attribution']//span[@class='c-byline__name']"

In [6]:
class Scraper:
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]


    query_extract_text =  QUERY_CONTENT
    query_extract_journalist = QUERY_CONTENT_JOURNALIST

    '''news_title = TITLE
    date = DATE'''

    def _init_(self):
        pass
    
    

    def clean_text(self, html_text):
        text_without_tags = w3lib.html.remove_tags(html_text)
        text_without_escape_chars = w3lib.html.replace_escape_chars(text_without_tags)
        text_without_escape_chars = html.unescape(text_without_escape_chars)
        text_without_whitespace = text_without_escape_chars.strip()
        return text_without_whitespace

    def replace_month_name_to_number(self,date_string):
        lower_string = date_string.lower()
        for month_id in range(1, 13):
            if(lower_string.find(calendar.month_name[month_id].lower())!=-1):
                if(month_id>=10):
                    return (lower_string.replace(calendar.month_name[month_id].lower(),str(month_id)))
                else:
                    return (lower_string.replace(calendar.month_name[month_id].lower(),"0"+str(month_id)))
            if(lower_string.find(calendar.month_abbr[month_id].lower())!=-1):
                if(month_id>=10):
                    return (lower_string.replace(calendar.month_abbr[month_id].lower(),str(month_id)))
                else:
                    return (lower_string.replace(calendar.month_abbr[month_id].lower(),"0"+str(month_id)))
                    
    #Se puede modificar
    def format_date(self,date):
        date = date.split("T")[0]
        return(date)

    #Se puede modificar
    def parse(self, response):
        self.news_text = []
        self.journalist = []
        self.message_to_send = []
        self.scraped_url = response.url
        
        print(response.html)
        
        news_text_extracted_response = response.html.xpath(self.query_extract_text) 
        journalist_extracted_response = response.html.xpath(self.query_extract_journalist) 
        
        
        for journalists in journalist_extracted_response:
            stripped_html_tags = w3lib.html.remove_tags(journalists.text)
            #print(stripped_html_tags)
            self.journalist.append(stripped_html_tags)

        
        for part_of_body in news_text_extracted_response:
            stripped_html_tags = w3lib.html.remove_tags(part_of_body.text)
            text_cleaned = self.clean_text(stripped_html_tags)
            self.news_text.append(text_cleaned)

    def start_request(self,url):
        headers = {'user-agent':random.choice(self.USER_AGENT_LIST) }
        session = HTMLSession()
        response = session.get(url,headers=headers)        
        return response      

In [ ]:
## el 10 de abajo deberia ser len(data)

In [16]:
for i in range (10):
    scraper = Scraper()
    print('obteniendo la noticia '+data[i]['title'])
    SEED_SCRAP = data[i]['url']
    response =scraper.start_request(SEED_SCRAP)
    scraper.parse(response)
    #falta añadir el nombre del periodsita y texto
    data[i]['text'] = scraper.news_text
    data[i]['journalist'] = scraper.journalist

obteniendo la noticia Verdict expected in trial of man who threw trailer hitch at Indigenous woman in Thunder Bay, Ont.
<HTML url='https://globalnews.ca/news/7519604/verdict-in-trial-of-man-who-threw-trailer-hitch-at-indigenous-woman-thunder-bay/'>
obteniendo la noticia Single-digit increases in COVID cases continue Sunday in Atlantic Canada
<HTML url='https://globalnews.ca/news/7519547/covid-cases-sunday-atlantic-canada/'>
obteniendo la noticia &#8216;Violent&#8217; arrest of Black SFU alumnus at Burnaby campus draws fire
<HTML url='https://globalnews.ca/news/7519374/violent-arrest-of-black-sfu-alumnus-at-burnaby-campus-draws-fire/'>
obteniendo la noticia Organizer of Kelowna’s anti-COVID-19 restriction protest fined: police
<HTML url='https://globalnews.ca/news/7519551/kelowna-covid-protest-organizer-fined/'>
obteniendo la noticia Alfonso Gagliano, former Liberal cabinet minister, dies at 78
<HTML url='https://globalnews.ca/news/7519550/alfonso-gagliano-dies-former-liberal-cabinet-mi

In [19]:
data[3]

{'url': 'https://globalnews.ca/?p=7519551',
 'title': 'Organizer of Kelowna’s anti-COVID-19 restriction protest fined: police',
 'date': '2020-12-13T19:23:05',
 'text': ['RCMP say they have fined an organizer of the large anti-COVID_19 restriction protest that marched through the downtown streets of Kelowna on Saturday.',
  '“We would like to remind the public that if they are engaging in activities contrary to the provincial public health order, they could be issued a ticket,” wrote Cpl. Jocelyn Noseworthy\xa0in an email.',
  '“At this time, I can advise that a ticket was issued to one of the organizers of this gathering for violating the provincial public health order.”',
  'The organizer self-identified themselves to police, which led to a ticket being issued for the fine of $2,300.',
  'Kelowna RCMP acknowledged that people in Canada have the democratic right to peaceful protest.',
  'However, police say it is up to their discretion to balance a citizen’s rights with the current po

In [20]:
# Almacenar los datos en un archivo json
with open('canada_global_news.json', 'w') as f:
    json.dump(data, f)